# Pystac Tutorial
## PySTAC Introduction

This tutorial includes a basic introduction on reading, writing, and creating STAC objects using Pystac.

It is adapted from the tutorials within the [sat-stac repo](https://github.com/sat-utils/sat-stac/blob/master/tutorial-1.ipynb).

It uses an example stac stored in the `example-catalog` directory along-side this notebook. The example stac has the following format:

```
example-catalog/
├── catalog.json
└── eo
    ├── catalog.json
    ├── landsat-8-l1
    │   ├── catalog.json
    │   └── item.json
    └── sentinel-2-l1c
        ├── catalog.json
        └── sentinel-2a
            ├── catalog.json
            └── item.json
```

In [4]:
import sys
sys.path.append('..')

In [74]:
from pystac import Catalog, Collection, Item

### Working with existing catalogs

Open a root catalog from it's json file

In [6]:
cat = Catalog.from_file('example-catalog/catalog.json')

We can see all elements of the STAC using the `describe` method

In [8]:
cat.describe()

* <Catalog id=stac>
/home/simon/files/rv/stac/pystac/tutorials/example-catalog/eo/catalog.json
    * <Catalog id=stac-eo>
/home/simon/files/rv/stac/pystac/tutorials/example-catalog/eo/sentinel-2-l1c/catalog.json
/home/simon/files/rv/stac/pystac/tutorials/example-catalog/eo/landsat-8-l1/catalog.json
        * <Collection id=sentinel-2-l1c>
/home/simon/files/rv/stac/pystac/tutorials/example-catalog/eo/sentinel-2-l1c/sentinel-2a/catalog.json
            * <Catalog id=sentinel-2a>
/home/simon/files/rv/stac/pystac/tutorials/example-catalog/eo/sentinel-2-l1c/sentinel-2a/item.json
              * <Item id=L1C_T53MNQ_A017245_20181011T011722>
        * <Collection id=landsat-8-l1>
/home/simon/files/rv/stac/pystac/tutorials/example-catalog/eo/landsat-8-l1/item.json
          * <Item id=LC08_L1GT_120046_20181012_20181012_01_RT>


Each STAC object has links that you can use to traverse the STAC tree

In [9]:
cat.links

[<Link rel=root target=<Catalog id=stac>>,
 <Link rel=child target=<Catalog id=stac-eo>>,
 <Link rel=self target=example-catalog/catalog.json>]

Pystac has several methods that allow you to access links:

In [15]:
# Get all child links
cat.get_child_links()

[<Link rel=child target=<Catalog id=stac-eo>>]

In [17]:
# Get a single link by 'rel'
cat.get_single_link('self')

<Link rel=self target=example-catalog/catalog.json>

In [18]:
# Get item links directly within this catalog (there are none for this catalog)
cat.get_item_links()

[]

or the items directly:

In [22]:
# get all child objects
cat.get_children()

[<Catalog id=stac-eo>]

In [23]:
# or a single child by id
cat.get_child('stac-eo')

<Catalog id=stac-eo>

In [20]:
# get all items anywhere below this catalog on the STAC tree
cat.get_all_items()

[<Item id=L1C_T53MNQ_A017245_20181011T011722>,
 <Item id=LC08_L1GT_120046_20181012_20181012_01_RT>]

You can access the stac item from a link using the `target` property

In [29]:
l = cat.get_single_link('child')
print(l)

<Link rel=child target=<Catalog id=stac-eo>>


In [30]:
print(l.target)

<Catalog id=stac-eo>


You can convert any stac item to a python dict using the `to_dict` method.

In [34]:
cat.to_dict()

{'id': 'stac',
 'stac_version': '0.8.0',
 'description': 'A STAC of public datasets',
 'links': [{'rel': 'root',
   'href': 'example-catalog/catalog.json',
   'type': 'application/json'},
  {'rel': 'child',
   'href': '/home/simon/files/rv/stac/pystac/tutorials/example-catalog/eo/catalog.json'},
  {'rel': 'self',
   'href': 'example-catalog/catalog.json',
   'type': 'application/json'}]}

In [35]:
# get first (and only in this case) sub-catalog
subcat = cat.get_children()[0]

In [36]:
# print some IDs
print("Root Catalog: ", cat.id)
print("Sub Catalog: ", subcat.id)
print("Sub Catalog parent: ", subcat.get_parent().id)

# iterate through child catalogs of the sub-catalog
print("Sub Catalog children:")
for child in subcat.get_children():
    print('    ', child.id)

Root Catalog:  stac
Sub Catalog:  stac-eo
Sub Catalog parent:  stac
Sub Catalog children:
     sentinel-2-l1c
     landsat-8-l1


In [39]:
print('\n**Items**')
for i in cat.get_all_items():
    print(i.id)


**Items**
L1C_T53MNQ_A017245_20181011T011722
LC08_L1GT_120046_20181012_20181012_01_RT


### Creating new catalogs

You can initialize a new Catalog with an id and a description. Note that by default it sets a new catalog as root.

In [49]:
# create a Catalog object with JSON
cat_json = {
    "id": "mycat",
    "description": "My shiny new STAC catalog"
}
mycat = Catalog(**cat_json)

In [50]:
mycat.links

[<Link rel=root target=<Catalog id=mycat>>]

### Adding catalogs to catalogs

In [51]:
# add a new catalog to a root catalog
kitten_json = {
    "id": "mykitten",
    "description": "A child catalog of my shiny new STAC catalog"
}

kitten = Catalog(**kitten_json)

When you add a child catalog to a parent catalog, the child catalog assumes the root catalog of it's parent. 'Child' and 'parent' links are also added to the parent and child catalogs, respectively.

In [52]:
kitten.links

[<Link rel=root target=<Catalog id=mykitten>>]

In [53]:
mycat.add_child(kitten)

In [54]:
kitten.links

[<Link rel=root target=<Catalog id=mycat>>,
 <Link rel=parent target=<Catalog id=mycat>>]

In [57]:
mycat.links

[<Link rel=root target=<Catalog id=mycat>>,
 <Link rel=child target=<Catalog id=mykitten>>]

In [55]:
mycat.describe()

* <Catalog id=mycat>
    * <Catalog id=mykitten>


### Adding collections to catalogs

In the next two steps we will work with Pystac Collections and Items. We will pull them out of our example catalog and add them to the new STAC that we have created.

Collections are Catalogs but also include spatial and temporal extents as well as addtitional properties. 

In [67]:
# open the Landsat collection
collection = Collection.from_file('example-catalog/eo/landsat-8-l1/catalog.json')
print('Collection name: ', collection.id)

Collection name:  landsat-8-l1


See the spatial and temporal extent of this collection

In [69]:
collection.extent.to_dict()

{'spatial': {'bbox': [[-180, -90, 180, 90]]},
 'temporal': {'interval': [['2013-06-01T00:00:00Z', None]]}}

In [70]:
collection.links

[<Link rel=root target=<Collection id=landsat-8-l1>>,
 <Link rel=root target=../../catalog.json>,
 <Link rel=parent target=../catalog.json>,
 <Link rel=item target=item.json>,
 <Link rel=self target=example-catalog/eo/landsat-8-l1/catalog.json>]

In [71]:
# add it to the child catalog created above
kitten.add_child(collection)

In [72]:
collection.links

[<Link rel=item target=item.json>,
 <Link rel=self target=example-catalog/eo/landsat-8-l1/catalog.json>,
 <Link rel=root target=<Catalog id=mycat>>,
 <Link rel=parent target=<Catalog id=mykitten>>]

### Adding items to collection

Items are stac objects whose parents can be either Catalogs or Collections. They also have spatio-temporal information and assets. Assets point directly to the data included in the STAC.

In [83]:
# open a Landsat item
item = Item.from_file('example-catalog/eo/sentinel-2-l1c/sentinel-2a/item.json')
print('Item name: ', item.id)

Item name:  L1C_T53MNQ_A017245_20181011T011722


In [84]:
item.links

[<Link rel=self target=item.json>,
 <Link rel=root target=../../../catalog.json>,
 <Link rel=parent target=catalog.json>,
 <Link rel=collection target=../catalog.json>]

In [80]:
item.assets

{'B01': <Asset href=https://sentinel-s2-l1c.s3.amazonaws.com/tiles/53/M/NQ/2018/10/11/0/B01.jp2>,
 'B02': <Asset href=https://sentinel-s2-l1c.s3.amazonaws.com/tiles/53/M/NQ/2018/10/11/0/B02.jp2>,
 'B03': <Asset href=https://sentinel-s2-l1c.s3.amazonaws.com/tiles/53/M/NQ/2018/10/11/0/B03.jp2>,
 'B04': <Asset href=https://sentinel-s2-l1c.s3.amazonaws.com/tiles/53/M/NQ/2018/10/11/0/B04.jp2>,
 'B05': <Asset href=https://sentinel-s2-l1c.s3.amazonaws.com/tiles/53/M/NQ/2018/10/11/0/B05.jp2>,
 'B06': <Asset href=https://sentinel-s2-l1c.s3.amazonaws.com/tiles/53/M/NQ/2018/10/11/0/B06.jp2>,
 'B07': <Asset href=https://sentinel-s2-l1c.s3.amazonaws.com/tiles/53/M/NQ/2018/10/11/0/B07.jp2>,
 'B08': <Asset href=https://sentinel-s2-l1c.s3.amazonaws.com/tiles/53/M/NQ/2018/10/11/0/B08.jp2>,
 'B09': <Asset href=https://sentinel-s2-l1c.s3.amazonaws.com/tiles/53/M/NQ/2018/10/11/0/B09.jp2>,
 'B10': <Asset href=https://sentinel-s2-l1c.s3.amazonaws.com/tiles/53/M/NQ/2018/10/11/0/B10.jp2>,
 'B11': <Asset href=

In [85]:
# add it to the collection created above
collection.add_item(item)

In [89]:
# now look at the catalog we've created
mycat.describe()

* <Catalog id=mycat>
    * <Catalog id=mykitten>
        * <Collection id=landsat-8-l1>
          * <Item id=LC08_L1GT_120046_20181012_20181012_01_RT>
        * <Collection id=landsat-8-l1>
          * <Item id=LC08_L1GT_120046_20181012_20181012_01_RT>


Currently, this STAC only exists in memory. We can specify the uri's using the `set_uris_from_root` method. You give this method a root directory and it will generate paths for all STAC objects relative to that directory.

In [91]:
mycat.set_uris_from_root('~/pystac-example/')
mycat.save()